In [1]:
# use tensorflow model trained from POS data:
path = "/Users/tollef/Downloads/git/PHD/SUMMARIZATION/grammaticality/src/POS-grammar-check/models/is_gram_20230512-141601_99acc.h5"
from tensorflow.keras.models import load_model
model = load_model(path)

Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



In [9]:
# load the nocola ungrammatical data:
path = "/Users/tollef/Downloads/git/PHD/SUMMARIZATION/grammaticality/src/data/nocola_parsed/nocola_ungrammatical_dev.txt"
with open(path, "r", encoding="utf-8") as f:
    ungrammatical = f.readlines()
ungrammatical = [line.split("\t")[0].strip() for line in ungrammatical]

In [10]:
ungrammatical[1]

'Alt dette kan påvirke helsen og livskvaliteten i negativ rettning.'

In [20]:
len(ungrammatical)
ungrammatical = ungrammatical[:100]  # subset

In [22]:
import spacy
import random
import numpy as np

nlp = spacy.load("nb_core_news_md")

s = "Frakoblingen skjedde ikke, og raketten begynte å spinne. Den spant i omtrent ett minutt før det kom en eksplosjon som ødela begge deler av raketten."

doc = nlp(s)

MAX_LEN = 34
pos_map = {
    'ADJ': 0,
    'ADP': 1,
    'ADV': 2,
    'AUX': 3,
    'CCONJ': 4,
    'DET': 5,
    'INTJ': 6,
    'NOUN': 7,
    'NUM': 8,
    'PART': 9,
    'PRON': 10,
    'PROPN': 11,
    'PUNCT': 12,
    'SCONJ': 13,
    'SYM': 14,
    'UNK': 15,
    'VERB': 16,
    'X': 17
}

def pred_from_pos(pos):
    pos = np.pad(pos, (0, MAX_LEN - len(pos)), constant_values="UNK")
    pos = np.vectorize(pos_map.get)(pos)
    pred = model.predict(np.asarray([pos]), verbose=0)[0][0]
    return pred

def pred_from_doc(doc):
    pos = [token.pos_ for token in doc]
    return pred_from_pos(pos)

def pred_from_text(text, nlp):
    doc = nlp(text)
    return pred_from_doc(doc)


def parse_pred(pred, threshold=0.08):
    # badly worded sentences will typically have a score of 0.001 or similar.
    res = "Grammatical" if pred > threshold else "Ungrammatical"
    confidence = pred * 100 if pred > threshold else (1 - pred) * 100
    return f"{res} ({confidence:.0f}% confidence)"

def shuffle_spacy_doc(doc):
    random_seed = 42
    # shuffle on the text and pos tags equally
    texts = [token.text for token in doc]
    pos = [token.pos_ for token in doc]
    random.Random(random_seed).shuffle(texts)
    random.Random(random_seed).shuffle(pos)
    return texts, pos


for text in ungrammatical:
    doc = nlp(text)
    for sent in doc.sents:
        pos = [token.pos_ for token in sent]
        # create 3 shuffled examples:
        print(f"{sent}\n--> {parse_pred(pred_from_pos(pos))}")
    
        

Likevel tror jeg ikke at det bare er på grunn av den lette tilgangen at handelen på Internett er så populært.
--> Grammatical (92% confidence)
Alt dette kan påvirke helsen og livskvaliteten i negativ rettning.
--> Grammatical (91% confidence)
Her passer det kanskje godt å kort diskutere om eliteutøvere i dag selv er ansvarlige for det kompromissløse resultatjaget, eller omsterke sponsorer og bakmenn må ta sin del av skylden.
--> Grammatical (91% confidence)
"Et smil er bruen til forståelse", slik lyder en kinesisk visdom oversatt på norsk.
--> Grammatical (92% confidence)
Derfor er det enklere og på denne måten fortere enn kollektivtransport.
--> Grammatical (35% confidence)
M Det krever tid og tålmodighet, og de to tingene er det vanskelig å finne hos unge gutter.
--> Grammatical (92% confidence)
Man kan se at utviklingen i mitt hjemland, samt i hele den vestlige verden har ført fram til et mer likestilt samfunn.
--> Grammatical (92% confidence)
Jeg håper at jeg har et så godt vennska

ValueError: index can't contain negative values

: 